# 7DaysOfCode: Dia 2 - Storytelling com Dados do CEAPS

Neste notebook, vamos explorar os dados do CEAPS (Cota para Exercício da Atividade Parlamentar dos Senadores) e criar uma narrativa visual através de gráficos. O objetivo é responder a perguntas e identificar padrões interessantes.

## 1. Carregamento dos Dados

Primeiro, carrego o conjunto de dados tratado anteriormente, que foi salvo em formato CSV.


In [1]:
import pandas as pd

# Carregar os dados tratados
ceaps_2019_2022 = pd.read_csv('../data/ceaps_2019_2022_tratado.csv', sep=';', decimal=',', encoding='ISO-8859-1')
ceaps_2019_2022.info()
print(ceaps_2019_2022.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69354 entries, 0 to 69353
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ANO                69354 non-null  int64  
 1   MES                69354 non-null  int64  
 2   SENADOR            69354 non-null  object 
 3   TIPO_DESPESA       69354 non-null  object 
 4   CNPJ_CPF           69354 non-null  object 
 5   FORNECEDOR         69354 non-null  object 
 6   DOCUMENTO          69342 non-null  object 
 7   DATA               69354 non-null  object 
 8   DETALHAMENTO       38583 non-null  object 
 9   VALOR_REEMBOLSADO  69354 non-null  float64
 10  COD_DOCUMENTO      69354 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 5.8+ MB
    ANO  MES       SENADOR                                       TIPO_DESPESA  \
0  2019    1  ACIR GURGACZ  Aluguel de imóveis para escritório político, c...   
1  2019    1  ACIR GURGACZ  Aluguel de imóv

## 2. Tratando data
- verifica-se que a data não está no formato correto.


In [2]:
ceaps_2019_2022['DATA'] = pd.to_datetime(ceaps_2019_2022['DATA'], format='%Y-%m-%d', errors='coerce')
print(ceaps_2019_2022['DATA'].isnull().sum())
ceaps_2019_2022.info()

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69354 entries, 0 to 69353
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ANO                69354 non-null  int64         
 1   MES                69354 non-null  int64         
 2   SENADOR            69354 non-null  object        
 3   TIPO_DESPESA       69354 non-null  object        
 4   CNPJ_CPF           69354 non-null  object        
 5   FORNECEDOR         69354 non-null  object        
 6   DOCUMENTO          69342 non-null  object        
 7   DATA               69354 non-null  datetime64[ns]
 8   DETALHAMENTO       38583 non-null  object        
 9   VALOR_REEMBOLSADO  69354 non-null  float64       
 10  COD_DOCUMENTO      69354 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 5.8+ MB


## 3. Análise Exploratória de Dados

Análise inicial dos dados para entender sua estrutura e identificar padrões gerais.
Começo verificando as colunas disponíveis e sua descrição estatística.

In [3]:
# Resumo estatístico das variáveis numéricas
print(ceaps_2019_2022.describe())

# Distribuição dos gastos por senador
gastos_por_senador = ceaps_2019_2022.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().sort_values(ascending=False)
gastos_por_senador.head()

                ANO           MES                           DATA  \
count  69354.000000  69354.000000                          69354   
mean    2020.415246      6.586412  2020-11-30 18:44:16.631196416   
min     2019.000000      1.000000            2019-01-01 00:00:00   
25%     2019.000000      4.000000            2019-10-16 00:00:00   
50%     2020.000000      7.000000            2020-12-04 00:00:00   
75%     2021.000000     10.000000            2021-12-14 00:00:00   
max     2022.000000     12.000000            2022-12-31 00:00:00   
std        1.162439      3.369575                            NaN   

       VALOR_REEMBOLSADO  COD_DOCUMENTO  
count       69354.000000   6.935400e+04  
mean         1413.102937   2.153677e+06  
min             0.010000   2.112846e+06  
25%           158.205000   2.134379e+06  
50%           426.660000   2.153704e+06  
75%          1570.390000   2.172931e+06  
max        120000.000000   2.199784e+06  
std          3097.720012   2.258380e+04  


SENADOR
TELMÁRIO MOTA       1951401.21
ROGÉRIO CARVALHO    1936315.87
MECIAS DE JESUS     1913241.64
PAULO ROCHA         1885315.28
MAILZA GOMES        1793120.58
Name: VALOR_REEMBOLSADO, dtype: float64

## 3. Storytelling com Visualizações

Criação de visualizações de gráficos Plotly para representação dos dados.



### 3.1 Senadores com Maiores Despesas Declaradas

10 senadores que mais declarou despesas.

In [4]:
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go

# Primeiro, agrupo as despesas por senador e somo os valores totais
despesas_por_senador = ceaps_2019_2022.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index().sort_values(by='VALOR_REEMBOLSADO', ascending=False).head(10)

# Criamos um gráfico de barras para visualizar as despesas de cada senador
trace = go.Bar(
    x=despesas_por_senador['SENADOR'],
    y=despesas_por_senador['VALOR_REEMBOLSADO'],
    marker=dict(color='skyblue')
)

layout = go.Layout(
    title='Despesas Declaradas por Senador (2019-2022) - Top 10 senadores com maiores gráficos',
    xaxis=dict(title='Senador'),
    yaxis=dict(title='Total de Despesas'),
    showlegend=False
)

fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)



### 3.2 Comparação de Despesas em Anos de Eleição e Não Eleitorais

Gastos de senadores em anos de eleição em comparação com anos não eleitorais.

In [5]:

# Adicionar uma coluna para indicar se é ano de eleição
#Lambda para verificar o único ano de eleição do dataset:  2022
ceaps_2019_2022['ANO_ELEICAO'] = ceaps_2019_2022['ANO'].apply(lambda x: 'Ano de Eleição' if x == 2022 else 'Ano Não Eleitoral')

# Agrupar as despesas por ano e tipo de ano
despesas_ano = ceaps_2019_2022.groupby(['ANO_ELEICAO', 'ANO'])['VALOR_REEMBOLSADO'].sum().reset_index()

# Criar gráficos para comparar os gastos em anos de eleição e não eleição
trace1 = go.Bar(
    x=despesas_ano[despesas_ano['ANO_ELEICAO'] == 'Ano de Eleição']['ANO'],
    y=despesas_ano[despesas_ano['ANO_ELEICAO'] == 'Ano de Eleição']['VALOR_REEMBOLSADO'],
    name='Ano de Eleição',
    marker=dict(color='lightcoral')
)

trace2 = go.Bar(
    x=despesas_ano[despesas_ano['ANO_ELEICAO'] == 'Ano Não Eleitoral']['ANO'],
    y=despesas_ano[despesas_ano['ANO_ELEICAO'] == 'Ano Não Eleitoral']['VALOR_REEMBOLSADO'],
    name='Ano Não Eleitoral',
    marker=dict(color='lightblue')
)

layout = go.Layout(
    title='Comparação de Despesas em Anos de Eleição e Não Eleitoral',
    xaxis=dict(title='Ano'),
    yaxis=dict(title='Total de Despesas'),
    barmode='group'
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
py.iplot(fig)


### 3.3 Despesa Declarada por Tipo de despesa

Tipos de despesas existentes e quais são as maiores.

In [6]:

# Agrupamos as despesas por tipo e somamos os valores
despesas_por_tipo = ceaps_2019_2022.groupby('TIPO_DESPESA')['VALOR_REEMBOLSADO'].sum().reset_index().sort_values(by='VALOR_REEMBOLSADO', ascending=False).head(10)

# Criando o gráfico
fig = go.Figure()

# Adicionando uma série de dados ao gráfico (gráfico de pizza)
fig.add_trace(go.Pie(
    labels=despesas_por_tipo['TIPO_DESPESA'],
    values=despesas_por_tipo['VALOR_REEMBOLSADO'],
    hole=0.3,  # Faz um gráfico de pizza com um buraco no meio (opcional)
    marker=dict(colors=['#D3B5E0', 'lightgreen', 'lightcoral', 'lightpink', 'lightgoldenrodyellow', 'lightblue', 'lightgray', 'lightyellow', 'lightcyan', 'lightsteelblue'])
))

# Configurando o layout
fig.update_layout(
    title='Despesas Declaradas por Tipo ',
    legend=dict(
        traceorder='normal',
        title='Tipos de Despesa',  # Adiciona um título à legenda
        itemclick='toggleothers'  # Permite clicar para alternar a visibilidade
    ),
    showlegend=False  # Exibe a legenda para identificar cada fatia
)

# Exibindo o gráfico
fig.show()

### 3.4 Analise temporal de despesas

Total das despesas durante os meses (2019 - 2022).

In [7]:

despesas_por_periodo = ceaps_2019_2022.groupby([ceaps_2019_2022['DATA'].dt.to_period('M')])['VALOR_REEMBOLSADO'].sum().reset_index()
despesas_por_periodo['DATA'] = despesas_por_periodo['DATA'].dt.to_timestamp()  # Convertendo o período para timestamp

# Criar gráfico de linha
fig = go.Figure()
fig.add_trace(go.Scatter(x=despesas_por_periodo['DATA'], y=despesas_por_periodo['VALOR_REEMBOLSADO'], mode='lines+markers'))

fig.update_layout(
    title='Evolução das Despesas ao Longo do Tempo',
    xaxis_title='Data',
    yaxis_title='Total de Despesas'
)

fig.show()


### 3.5 Fornecedores mais ganhos

10 fornecedores com maiores investimentos.

In [8]:
# Primeiro, agrupamos as despesas por senador e somamos os valores
despesas_por_fornecedor = ceaps_2019_2022.groupby('FORNECEDOR')['VALOR_REEMBOLSADO'].sum().reset_index().sort_values(by='VALOR_REEMBOLSADO', ascending=False).head(10)

# Criamos um gráfico de barras para visualizar as despesas de cada senador
trace = go.Bar(
    x=despesas_por_fornecedor['FORNECEDOR'],
    y=despesas_por_fornecedor['VALOR_REEMBOLSADO'],
    marker=dict(color='#D3B5E0')
)

layout = go.Layout(
    title='Despesas Declaradas por Fornecedor (2019-2022) - Top 10 fornececores',
    xaxis=dict(title='Fornecedor'),
    yaxis=dict(title='Total de Despesas'),
    showlegend=False
)

fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)

## 5. Conclusão

### Conclusão

Neste notebook, exploramos os dados do CEAPS e criamos visualizações interativas para responder perguntas chave sobre as despesas dos senadores.